# Imports

In [1]:
import nltk

# Constants

# Grammar-Based Feature Extraction

## Context-Free Grammars

In [2]:
GRAMMAR = """
    S -> NNP VP
    VP -> V PP
    PP -> P NP
    NP -> DT N
    NNP -> 'Gwen' | 'George'
    V -> 'looks' | 'burns'
    P -> 'in' | 'for'
    DT -> 'the'
    N -> 'castle' | 'ocean'
"""

In [3]:
cfg = nltk.CFG.fromstring(GRAMMAR)

In [4]:
print(cfg)

Grammar with 13 productions (start state = S)
    S -> NNP VP
    VP -> V PP
    PP -> P NP
    NP -> DT N
    NNP -> 'Gwen'
    NNP -> 'George'
    V -> 'looks'
    V -> 'burns'
    P -> 'in'
    P -> 'for'
    DT -> 'the'
    N -> 'castle'
    N -> 'ocean'


In [5]:
cfg.start()

S

In [6]:
cfg.productions()

[S -> NNP VP,
 VP -> V PP,
 PP -> P NP,
 NP -> DT N,
 NNP -> 'Gwen',
 NNP -> 'George',
 V -> 'looks',
 V -> 'burns',
 P -> 'in',
 P -> 'for',
 DT -> 'the',
 N -> 'castle',
 N -> 'ocean']

## Syntactic Parsers

## Extracting Keyphrases

## Extracting Entities

# n-Gram Feature Extraction

## An n-Gram-Aware CorpusReader

## Choosing the Right n-Gram Window

## Significant Collocations

# n-Gram Language Models

## Frequency and Conditional Frequency

## Estimating Maximum Likelihood

## Unknown Words: Back-off and Smoothing

## Language Generation